# Team_8:Movies Recommender System  Solution

# Table of Contents
    1. Introduction
    2. Initializing Comet
    3. Importing Libraries 
    4. Loading dataset
    5. Exploratory Data Analysis
    6. Data Preprocessing
    7. Modelling
    8. Hyperparameter Tuning
    9. Creating Kaggle Submission
    10. Result and Discussion
    11. Conclusion

# Introduction

# Initializing Comet

In [2]:
# import comet_ml at the top of your file
from comet_ml import Experiment

# Create an experiment with your api key
experiment = Experiment(
    api_key="iCbPjOAd54YXBHeiHBB2EY9YQ",
    project_name="2110acds-t8-movie-recommender",
    workspace="nic-py",
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. You can override where Comet is looking for a Git Patch by setting the configuration `COMET_GIT_DIRECTORY`
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/nic-py/2110acds-t8-movie-recommender/97d4486dac0c4953a3abcf878984336a



# Installing Packages

In [3]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        
        
import warnings
warnings.filterwarnings('ignore')

/kaggle/input/edsa-movie-recommendation-wilderness/sample_submission.csv
/kaggle/input/edsa-movie-recommendation-wilderness/movies.csv
/kaggle/input/edsa-movie-recommendation-wilderness/imdb_data.csv
/kaggle/input/edsa-movie-recommendation-wilderness/genome_tags.csv
/kaggle/input/edsa-movie-recommendation-wilderness/genome_scores.csv
/kaggle/input/edsa-movie-recommendation-wilderness/train.csv
/kaggle/input/edsa-movie-recommendation-wilderness/test.csv
/kaggle/input/edsa-movie-recommendation-wilderness/tags.csv
/kaggle/input/edsa-movie-recommendation-wilderness/links.csv


In [9]:
train = pd.read_csv('../input/edsa-movie-recommendation-wilderness/train.csv')
genome_scores = pd.read_csv('../input/edsa-movie-recommendation-wilderness/genome_scores.csv')
genome_tags =pd.read_csv('../input/edsa-movie-recommendation-wilderness/genome_tags.csv')
imdb = pd.read_csv('../input/edsa-movie-recommendation-wilderness/imdb_data.csv')
links = pd.read_csv('../input/edsa-movie-recommendation-wilderness/links.csv')
movies = pd.read_csv('../input/edsa-movie-recommendation-wilderness/movies.csv')
tags = pd.read_csv('../input/edsa-movie-recommendation-wilderness/tags.csv')
test = pd.read_csv('../input/edsa-movie-recommendation-wilderness/test.csv')
sample_submision = pd.read_csv('../input/edsa-movie-recommendation-wilderness/sample_submission.csv')

<a id="three"></a>
## 3. Exploratory Data Analysis (EDA)
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Exploratory data analysis ⚡ |
| :--------------------------- |
| In this section, we perform an in-depth analysis of all the variables in the DataFrame. |

---


In [5]:
train.head()

,userId,movieId,rating,timestamp
0,5163,57669,4.0,1518349992
1,106343,5,4.5,1206238739
2,146790,5459,5.0,1076215539
3,106362,32296,2.0,1423042565
4,9041,366,3.0,833375837


In [6]:
train.describe()

,userId,movieId,rating,timestamp
count,1.000004e+07,1.000004e+07,1.000004e+07,1.000004e+07
mean,8.119909e+04,2.138911e+04,3.533395e+00,1.215677e+09
std,4.679359e+04,3.919578e+04,1.061124e+00,2.268921e+08
min,1.000000e+00,1.000000e+00,5.000000e-01,7.896520e+08
25%,4.051000e+04,1.197000e+03,3.000000e+00,1.011742e+09
50%,8.091400e+04,2.947000e+03,3.500000e+00,1.199019e+09
75%,1.215790e+05,8.630000e+03,4.000000e+00,1.447242e+09
max,1.625410e+05,2.091710e+05,5.000000e+00,1.574328e+09


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000038 entries, 0 to 10000037
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 305.2 MB


<a id="four"></a>
## 4. Data Preprocessing
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Data Preprocessing ⚡ |
| :--------------------------- |
| Data Preparation is one of the indispensable steps in any Machine Learning development life cycle. In today’s world, the data is present in a structured as well as unstructured form. To deal with such data, data scientists spent almost 70–80% of their time in preparing data for further analysis. |

---

<a id="five"></a>
## 5. Modelling
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Modelling ⚡ |
| :--------------------------- |
| In this section, we are required to create recommender systems. |

---

In [8]:
experiment.end()

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/nic-py/2110acds-t8-movie-recommender/97d4486dac0c4953a3abcf878984336a
COMET INFO:   Uploads:
COMET INFO:     conda-info          : 1
COMET INFO:     conda-specification : 1
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 1
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: ---------------------------
COMET INFO: Uploading metrics, params, and assets to Comet before program termination (may take several seconds)
COMET INFO: The Python SDK has 3600 seconds to finish before aborting...
